In [47]:
import sqlite3
import pandas as pd

# 1. BAĞLANTI OLUŞTURMA
connection = sqlite3.connect("universite_obs.db")
cursor = connection.cursor()
cursor.execute("PRAGMA foreign_keys = ON;") # İlişkisel bütünlüğü aç

# ---------------------------------------------------------
# 2. TABLOLARI OLUŞTURMA (Hiyerarşik Sırayla)
# ---------------------------------------------------------
# Önce çocukları, sonra babaları siliyoruz (Hata almamak için)
tablolar = ["donemdersleri", "dersler", "ogrenciler", "hocalar", "bolumler", "fakulteler"]
for tablo in tablolar:
    cursor.execute(f"DROP TABLE IF EXISTS {tablo}")

# 1. Fakülteler (En üst katman)
cursor.execute("""
CREATE TABLE fakulteler (
    id INTEGER PRIMARY KEY,
    fakulteadi TEXT
);
""")

# 2. Bölümler (Fakülteye bağlı)
cursor.execute("""
CREATE TABLE bolumler (
    id INTEGER PRIMARY KEY,
    bolumadi TEXT,
    fakulteid INTEGER,
    FOREIGN KEY(fakulteid) REFERENCES fakulteler(id)
);
""")

# 3. Hocalar (Bölüme bağlı)
cursor.execute("""
CREATE TABLE hocalar (
    id INTEGER PRIMARY KEY,
    hocaadi TEXT,
    bolumid INTEGER,
    FOREIGN KEY(bolumid) REFERENCES bolumler(id)
);
""")

# 4. Dersler (Bölüme ve Hocaya bağlı)
cursor.execute("""
CREATE TABLE dersler (
    id INTEGER PRIMARY KEY,
    dersadi TEXT,
    bolumid INTEGER,
    hocaid INTEGER,
    FOREIGN KEY(bolumid) REFERENCES bolumler(id),
    FOREIGN KEY(hocaid) REFERENCES hocalar(id)
);
""")

# 5. Öğrenciler (Bölüme bağlı)
cursor.execute("""
CREATE TABLE ogrenciler (
    id INTEGER PRIMARY KEY,
    adsoyad TEXT,
    bolumid INTEGER,
    dogumt TEXT, -- Tarih formatı: YYYY-AA-GG
    FOREIGN KEY(bolumid) REFERENCES bolumler(id)
);
""")

# 6. Dönem Dersleri / Notlar (Öğrenci ve Derse bağlı - Kesişim Tablosu)
cursor.execute("""
CREATE TABLE donemdersleri (
    id INTEGER PRIMARY KEY,
    ogrenciid INTEGER,
    dersid INTEGER,
    notu INTEGER,
    FOREIGN KEY(ogrenciid) REFERENCES ogrenciler(id),
    FOREIGN KEY(dersid) REFERENCES dersler(id)
);
""")

print("Tüm tablolar başarıyla oluşturuldu.")

# ---------------------------------------------------------
# 3. VERİ EKLEME (INSERT)
# ---------------------------------------------------------

# Fakülteler
cursor.executemany("INSERT INTO fakulteler VALUES (?,?)", [
    (1, 'Mühendislik Fakültesi'),
    (2, 'Fen Edebiyat Fakültesi')
])

# Bölümler
cursor.executemany("INSERT INTO bolumler VALUES (?,?,?)", [
    (10, 'Bilgisayar Müh.', 1),  # Mühendislik'e bağlı
    (11, 'Endüstri Müh.', 1),    # Mühendislik'e bağlı
    (12, 'Matematik', 2)         # Fen Ed.'e bağlı
])

# Hocalar
cursor.executemany("INSERT INTO hocalar VALUES (?,?,?)", [
    (100, 'Prof. Dr. Ali Yılmaz', 10),  # Bilgisayar hocası
    (101, 'Doç. Dr. Ayşe Kaya', 10),     # Bilgisayar hocası
    (102, 'Dr. Mehmet Demir', 12)        # Matematik hocası
])

# Dersler
cursor.executemany("INSERT INTO dersler VALUES (?,?,?,?)", [
    (500, 'Algoritma', 10, 100),       # Bilgisayar Dersi, Ali Hoca
    (501, 'Veritabanı', 10, 101),      # Bilgisayar Dersi, Ayşe Hoca
    (502, 'Lineer Cebir', 12, 102)     # Matematik Dersi, Mehmet Hoca
])

# Öğrenciler
cursor.executemany("INSERT INTO ogrenciler VALUES (?,?,?,?)", [
    (1, 'Ahmet Hakan', 10, '2002-05-15'),  # Bilgisayar öğr.
    (2, 'Zeynep Su', 10, '2001-08-20'),    # Bilgisayar öğr.
    (3, 'Cemil Can', 12, '2003-01-10')     # Matematik öğr.
])

# Notlar (Dönem Dersleri)
cursor.executemany("INSERT INTO donemdersleri VALUES (?,?,?,?)", [
    (1, 1, 500, 85),  # Ahmet, Algoritma'dan 85 aldı
    (2, 1, 501, 40),  # Ahmet, Veritabanı'ndan 40 aldı
    (3, 2, 500, 90),  # Zeynep, Algoritma'dan 90 aldı
    (4, 3, 502, 75)   # Cemil, Lineer Cebir'den 75 aldı
])

connection.commit()
print("Veriler başarıyla eklendi.")

# ---------------------------------------------------------
# 4. KONTROL SORGUSU (Test)
# ---------------------------------------------------------
# Tüm ilişkileri test eden "Büyük Resim" sorgusu
sql_test = """
SELECT 
    f.fakulteadi,
    b.bolumadi,
    h.hocaadi,
    d.dersadi,
    o.adsoyad,
    n.notu
FROM donemdersleri n
JOIN ogrenciler o ON n.ogrenciid = o.id
JOIN dersler d ON n.dersid = d.id
JOIN hocalar h ON d.hocaid = h.id
JOIN bolumler b ON o.bolumid = b.id
JOIN fakulteler f ON b.fakulteid = f.id;
"""

print("-" * 50)
print("SİSTEM ÖZETİ RAPORU:")
df = pd.read_sql(sql_test, connection)
print(df)

def run_query(query):
    try:
        result = pd.read_sql_query(query, connection)
        print(result)
    except Exception as e:
        print(f"Sorgu çalıştırılırken hata oluştu: {e}")
        return None



Tüm tablolar başarıyla oluşturuldu.
Veriler başarıyla eklendi.
--------------------------------------------------
SİSTEM ÖZETİ RAPORU:
               fakulteadi         bolumadi               hocaadi  \
0   Mühendislik Fakültesi  Bilgisayar Müh.  Prof. Dr. Ali Yılmaz   
1   Mühendislik Fakültesi  Bilgisayar Müh.    Doç. Dr. Ayşe Kaya   
2   Mühendislik Fakültesi  Bilgisayar Müh.  Prof. Dr. Ali Yılmaz   
3  Fen Edebiyat Fakültesi        Matematik      Dr. Mehmet Demir   

        dersadi      adsoyad  notu  
0     Algoritma  Ahmet Hakan    85  
1    Veritabanı  Ahmet Hakan    40  
2     Algoritma    Zeynep Su    90  
3  Lineer Cebir    Cemil Can    75  


In [48]:
#ogrenciler tablosuna tcno string11 column ekle
cursor.execute("""
    alter table ogrenciler
    add column tcno char(11);
""")

In [49]:
#ogrenciler tablosunun dogumt alanını sil
cursor.execute("""
    alter table ogrenciler
    drop column dogumt;
""")

In [50]:
#müh fak ne ait ogrenci sayısını bölümadı listeleyen sql komut
run_query("""
    SELECT 
        b.bolumadi, 
        COUNT(o.id) as ogrenci_sayisi
    FROM bolumler b
    JOIN fakulteler f ON b.fakulteid = f.id
    LEFT JOIN ogrenciler o ON b.id = o.bolumid
    WHERE f.fakulteadi LIKE '%Mühendislik%'
    GROUP BY b.bolumadi;
""")

          bolumadi  ogrenci_sayisi
0  Bilgisayar Müh.               2
1    Endüstri Müh.               0


In [51]:
#fizik dersini alıp matematik dersini almayan ögrenci sayısı
run_query("""
    SELECT COUNT(DISTINCT ogrenciid) 
    FROM donemdersleri 
    WHERE dersid = (SELECT id FROM dersler WHERE dersadi = 'Fizik')
    AND ogrenciid NOT IN (
        SELECT ogrenciid 
        FROM donemdersleri 
        WHERE dersid = (SELECT id FROM dersler WHERE dersadi = 'Matematik')
);
""")

   COUNT(DISTINCT ogrenciid)
0                          0


In [52]:
#atilla ergüzen isimli hocanın ögrenci sayısını bulunuz 
run_query("""
    SELECT COUNT(DISTINCT dd.ogrenciid) as toplam_ogrenci_sayisi
    FROM hocalar h
    JOIN dersler d ON h.id = d.hocaid
    JOIN donemdersleri dd ON d.id = dd.dersid
    WHERE h.hocaadi = 'Atilla Ergüzen';
""")

   toplam_ogrenci_sayisi
0                      0


In [53]:
#en fazla ogrencisi olan hocanın adını bulunuz
run_query("""
    SELECT h.hocaadi, COUNT(DISTINCT dd.ogrenciid) as ogrenci_sayisi
    FROM hocalar h
    JOIN dersler d ON h.id = d.hocaid
    JOIN donemdersleri dd ON d.id = dd.dersid
    GROUP BY h.hocaadi
    ORDER BY ogrenci_sayisi DESC
    LIMIT 1;
""")

                hocaadi  ogrenci_sayisi
0  Prof. Dr. Ali Yılmaz               2


In [54]:
#attila ergüzenin en faz ogrencisi olan dersinin ortalamadan fazla not alan ogrenci sayısı 
run_query("""
    SELECT COUNT(*) 
    FROM donemdersleri
    WHERE dersid = (
        -- 1. ADIM: Atilla Hoca'nın en kalabalık dersinin ID'sini bul
        SELECT d.id
        FROM dersler d
        JOIN hocalar h ON d.hocaid = h.id
        JOIN donemdersleri dd ON d.id = dd.dersid
        WHERE h.hocaadi = 'Atilla Ergüzen'
        GROUP BY d.id
        ORDER BY COUNT(dd.ogrenciid) DESC
        LIMIT 1
    )
    AND notu > (
        -- 2. ADIM: O dersin ortalamasını bul
        SELECT AVG(notu)
        FROM donemdersleri
        WHERE dersid = (
            -- (Aynı alt sorgu tekrar edilir)
            SELECT d.id
            FROM dersler d
            JOIN hocalar h ON d.hocaid = h.id
            JOIN donemdersleri dd ON d.id = dd.dersid
            WHERE h.hocaadi = 'Atilla Ergüzen'
            GROUP BY d.id
            ORDER BY COUNT(dd.ogrenciid) DESC
            LIMIT 1
        )
    );
""")

   COUNT(*)
0         0


In [55]:
#ders kaydı yaptırmayan ogrencileri listeleyiniz
run_query("""
    SELECT * FROM ogrenciler 
    WHERE id NOT IN (SELECT DISTINCT ogrenciid FROM donemdersleri);
""")

Empty DataFrame
Columns: [id, adsoyad, bolumid, tcno]
Index: []


In [56]:
#ogrenciler tablosundan foreign primary key kaldır


In [57]:
connection.close()